In [1]:
import pulp
import pandas as pd
from pathlib import Path
import preprocess_data as ppd

In [2]:
data_path = Path("../data/")
output_path = Path("../output/")
df, adjacent_matrix = ppd.get_df_adj(data_path, 2021)
prob = pulp.LpProblem("model1", pulp.LpMinimize)

In [3]:
n = 88 + 1

In [4]:
x = pulp.LpVariable.dicts("x", range(1, n), cat="Binary")
y = pulp.LpVariable.dicts("y", range(1, n), cat="Binary")
a = pulp.LpVariable.dicts("a", (x, y), range(1, n), cat="Binary")

for i in range(1, n):
    prob += (
        pulp.lpSum([int(j in adjacent_matrix[i]) * x[j] for j in range(1, n)]) >= 1,
        f"constraint_a{i}",
    )

prob += (
    pulp.lpSum([x[i] for i in range(1, n)]),
    "objective_x",
)

In [5]:
prob.solve()
pulp.value(prob.objective)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/adb/.pyenv/versions/3.11.0/envs/ise-533/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/6cd6083a5c0040278dd6e6372e307114-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/6cd6083a5c0040278dd6e6372e307114-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 93 COLUMNS
At line 906 RHS
At line 995 BOUNDS
At line 1084 ENDATA
Problem MODEL has 88 rows, 88 columns and 548 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 14.8333 - 0.00 seconds
Cgl0004I processed model has 88 rows, 88 columns (88 integer (88 of which binary)) and 548 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 51 integers unsatisfied sum - 14.0833
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 15 iterations 44
Cbc0038I Solution found of 15
Cbc0038I Before mini

15.0

In [6]:
for x in prob.variables():
    if x.varValue != 0:
        print(x.name, "=", x.varValue)

x_28 = 1.0
x_3 = 1.0
x_35 = 1.0
x_40 = 1.0
x_45 = 1.0
x_49 = 1.0
x_5 = 1.0
x_51 = 1.0
x_7 = 1.0
x_72 = 1.0
x_75 = 1.0
x_76 = 1.0
x_8 = 1.0
x_81 = 1.0
x_9 = 1.0


In [7]:
chen = pd.read_csv(data_path / "chen_adj_mat.csv", header=None)
chen.loc[2].dropna()

1     1.0
2     1.0
3     1.0
16    1.0
Name: 2, dtype: float64

In [8]:
chen_adj_matrix = {}
for column in chen.columns:
    chen_adj_matrix[column + 1] = [
        i + 1 for i in (chen.loc[column].dropna().index.values.tolist())
    ]
chen_adj_matrix

{1: [1, 2, 18, 19],
 2: [1, 2, 3, 17, 18],
 3: [2, 3, 4, 17],
 4: [3, 4, 5, 15, 16, 17],
 5: [4, 5, 6, 15],
 6: [5, 6, 7, 12, 13, 14, 15],
 7: [6, 7, 8, 12],
 8: [7, 8, 9, 11, 12],
 9: [8, 9, 10, 11],
 10: [9, 10, 11, 28, 29],
 11: [8, 9, 10, 11, 12, 27, 28],
 12: [6, 7, 8, 11, 12, 13, 26, 27],
 13: [6, 12, 13, 14, 25, 26],
 14: [6, 13, 14, 15, 24, 25],
 15: [4, 5, 6, 14, 15, 16, 23, 24],
 16: [4, 15, 16, 17, 22, 23],
 17: [2, 3, 4, 16, 17, 18, 22],
 18: [1, 2, 17, 18, 19, 20, 21],
 19: [1, 18, 19, 20],
 20: [18, 19, 20, 21, 39],
 21: [18, 20, 21, 22, 37, 38, 39],
 22: [16, 17, 21, 22, 23, 36, 37],
 23: [15, 16, 22, 23, 24, 34, 35, 36],
 24: [14, 15, 23, 24, 25, 34],
 25: [13, 14, 24, 25, 26, 33, 34],
 26: [12, 13, 25, 26, 27, 33],
 27: [11, 12, 26, 27, 28, 32, 33],
 28: [10, 11, 27, 28, 29, 31, 32],
 29: [10, 28, 29, 30, 31],
 30: [29, 30, 31, 48, 49],
 31: [28, 29, 30, 31, 32, 47, 48],
 32: [27, 28, 31, 32, 33, 46, 47],
 33: [25, 26, 27, 32, 33, 34, 45, 46],
 34: [23, 24, 25, 33, 34,

In [9]:
# check adj matrix differences
df, my_camm_output = ppd.get_df_adj(data_path, 2021, camm=True)
for i in sorted(my_camm_output.keys()):
    check = set(my_camm_output[i]) == set(chen_adj_matrix[i])
    if not check:
        print("_______")
        county_index = df[df["camm_id"] == i].county_id.values.item()
        print(f"{i} me:  {my_camm_output[i]}\n   chen:{chen_adj_matrix[i]}")
        print(
            f"{county_index} me:  {ppd.camm_ids_to_county_ids(df, my_camm_output[i])}\n   chen:{ppd.camm_ids_to_county_ids(df, chen_adj_matrix[i])}"
        )

_______
17 me:  [2, 3, 4, 16, 17, 18, 21, 22]
   chen:[2, 3, 4, 16, 17, 18, 22]
69 me:  [2, 20, 32, 35, 63, 69, 81, 87]
   chen:[2, 20, 32, 35, 63, 69, 81]
_______
18 me:  [1, 2, 17, 18, 19, 20, 21, 22]
   chen:[1, 2, 17, 18, 19, 20, 21]
35 me:  [20, 26, 32, 35, 48, 69, 86, 87]
   chen:[20, 26, 35, 48, 69, 86, 87]
_______
21 me:  [17, 18, 20, 21, 22, 37, 38, 39]
   chen:[18, 20, 21, 22, 37, 38, 39]
87 me:  [32, 35, 48, 62, 69, 72, 74, 87]
   chen:[32, 35, 48, 62, 72, 74, 87]
_______
22 me:  [16, 17, 18, 21, 22, 23, 36, 37]
   chen:[16, 17, 21, 22, 23, 36, 37]
32 me:  [2, 32, 33, 35, 69, 74, 87, 88]
   chen:[2, 32, 33, 69, 74, 87, 88]
_______
60 me:  [58, 60, 61, 76, 77, 78, 79]
   chen:[58, 60, 61, 76, 78, 79]
18 me:  [18, 28, 43, 47, 52, 67, 77]
   chen:[18, 28, 43, 47, 52, 77]
_______
77 me:  [60, 75, 76, 77, 78, 81, 82]
   chen:[75, 76, 77, 78, 81, 82]
67 me:  [18, 28, 50, 67, 76, 77, 78]
   chen:[28, 50, 67, 76, 77, 78]
